In [ ]:

import csv
import firebase_admin
from firebase_admin import credentials, firestore


# Initialize Firebase Admin SDK
cred = credentials.Certificate("ServiceAccountKey.json")
firebase_admin.initialize_app(cred)

In [2]:
db = firestore.client()

In [3]:
csv_file_path = "Large_news_dataset.csv"  # Replace with your CSV file path
collection_name = "articles"

In [4]:
import pandas as pd

In [39]:
data = pd.read_csv("large_news_dataset_modified.csv")

In [40]:
data.head()

,title,summary,image_url,published,url,images
0,PM under pressure to respond to row over Lee A...,The PM is to face BBC interviews this morning ...,https://ichef.bbci.co.uk/ace/standard/240/cpsp...,"Mon, 26 Feb 2024 06:52:38 GMT",https://www.bbc.co.uk/news/live/uk-68400238,IMAGE_1.jpg
1,What’s gone wrong at Royal Mail?,"Lost letters, big losses and calls for reform:...",https://ichef.bbci.co.uk/ace/standard/240/cpsp...,"Mon, 26 Feb 2024 00:01:07 GMT",https://www.bbc.co.uk/news/business-68382286,IMAGE_2.jpg
2,"More young people out of work due to health, s...",A Resolution Foundation report is calling for ...,https://ichef.bbci.co.uk/ace/standard/240/cpsp...,"Mon, 26 Feb 2024 04:57:32 GMT",https://www.bbc.co.uk/news/health-68399392,IMAGE_3.jpg
3,Rail services in South East hit by signal failure,Thameslink Govia says it is unable to run any ...,https://ichef.bbci.co.uk/ace/standard/240/cpsp...,"Mon, 26 Feb 2024 07:01:21 GMT",https://www.bbc.co.uk/news/uk-england-sussex-6...,IMAGE_4.jpg
4,Diverse areas face car insurance 'ethnicity' bill,BBC Verify finds higher quotes in parts of Eng...,https://ichef.bbci.co.uk/ace/standard/240/cpsp...,"Mon, 26 Feb 2024 00:29:05 GMT",https://www.bbc.co.uk/news/business-68349396,IMAGE_5.jpg


In [13]:
doc_ref = db.collection(collection_name)

In [41]:
data.columns

Index(['title', 'summary', 'image_url', 'published', 'url', 'images'], dtype='object')

In [7]:
from datetime import datetime
import firebase_admin
import pytz

In [43]:
data['published'][175]

'Mon, 26 Feb 2024 07:34:37 '

In [44]:
def convert_to_firestore_timestamp(time_string):
    """
    Convert a GMT time string to a Firestore timestamp
    Args:
        time_string (str): Time string in format 'Mon, 26 Feb 2024 06:52:38 GMT'
    Returns:
        google.cloud.firestore.SERVER_TIMESTAMP: Firestore timestamp object
    """
    # Parse the time string to datetime object
    try:
        dt = datetime.strptime(time_string, '%a, %d %b %Y %H:%M:%S GMT')
    except:
        dt = datetime.strptime(time_string + 'GMT', '%a, %d %b %Y %H:%M:%S GMT')
    
    # Ensure the datetime is timezone-aware (GMT/UTC)
    dt = pytz.timezone('GMT').localize(dt)
    
    # Return the datetime object directly - Firestore will convert it automatically
    return dt

In [46]:
time_string = 'Mon, 26 Feb 2024 06:52:38 '
firestore_timestamp = convert_to_firestore_timestamp(time_string)
print(firestore_timestamp)

2024-02-26 06:52:38+00:00


In [ ]:
for i in range(len(data)):
    try:
        doc_ref.add({    
            'title': data['title'][i],
            'content': data['summary'][i],
            'published_at': convert_to_firestore_timestamp(data['published'][i]),
            'url': data['image_url'][i],
            'sentiment': ""
        })
    except:
        print("Error at index: ", i)

Error at index:  5390
Error at index:  5391
Error at index:  5900
Error at index:  5901
Error at index:  5902
Error at index:  5903
Error at index:  5904
Error at index:  5905
Error at index:  5906
Error at index:  5907
Error at index:  5908
Error at index:  5909
Error at index:  5910
Error at index:  5911
Error at index:  5912
Error at index:  5913
Error at index:  5914
Error at index:  5915
Error at index:  5916
Error at index:  5917
Error at index:  5918
Error at index:  5919
Error at index:  6597
Error at index:  6598
Error at index:  6599
Error at index:  6600
Error at index:  6601
Error at index:  6602
Error at index:  6603
Error at index:  6604
Error at index:  6605
Error at index:  6606
Error at index:  6607
Error at index:  6608
Error at index:  6609
Error at index:  6610
Error at index:  6611
Error at index:  6612
Error at index:  6613
Error at index:  6614
Error at index:  6615
Error at index:  6616
Error at index:  6617
Error at index:  6618
Error at index:  7332
Error at i

In [50]:
import requests

In [68]:
small_data.iloc[3].title

'Rail services in South East hit by signal failure'

In [69]:
url = "http://35.94.30.116:8000/generate"
headers = {'Content-Type': 'application/json'}
payload = {"news_article": small_data.iloc[4].title + " " + small_data.iloc[4].summary}

In [70]:
response = requests.post(
        url=url,
        headers=headers,
        json=payload,  # requests will automatically convert dict to JSON
        timeout=30  # timeout in seconds
    )



In [71]:
response.json()

{'sentiment': 'Negative\n\nThe article has an overall negative sentiment as it highlights potential discrimination in the car insurance industry against residents of ethnically diverse communities who face much higher premiums through no direct fault of their own. The investigation revealing these disparities prompts concerns about racial profiling, indirect discrimination and lack of fairness in insurance pricing.',
 'summary': "A BBC investigation found evidence of higher car insurance premiums in ethnically diverse areas of England, even when controlling for crime rates and accident statistics. In some neighborhoods, premiums were over £1,000 higher than nearby areas with less ethnic diversity. The investigation analyzed over 31 million quotes across England and revealed disparities in areas like Bradford, Luton and parts of London. Critics argue this amounts to racial profiling and indirect discrimination based on ethnicity and socioeconomic status. The Financial Conduct Authority 